# Drug2cell comparison

In [1]:
import warnings

warnings.filterwarnings("ignore")

## Original drug2cell implementation

In [2]:
import blitzgsea as blitz
import drug2cell as d2c
import scanpy as sc

adata = sc.datasets.pbmc3k_processed()

d2c.score(adata)

targets = blitz.enrichr.get_library("GO_Molecular_Function_2021")
sc.tl.rank_genes_groups(adata, method="wilcoxon", groupby="louvain", use_raw=False)
overrepresentation_old = d2c.hypergeometric(adata, targets=targets)

targets = blitz.enrichr.get_library("GO_Molecular_Function_2021")
enrichment_old, plot_gsea_args = d2c.gsea(adata, targets=targets)

## Pertpy implementation

In [3]:
import pertpy as pt

adata = sc.datasets.pbmc3k_processed()

pt_chembl = pt.md.Drug()
pt_chembl.annotate(adata, source="chembl")

pt_enricher = pt.tl.Enrichment()
pt_enricher.score(adata)

targets = blitz.enrichr.get_library("GO_Molecular_Function_2021")
sc.tl.rank_genes_groups(adata, method="wilcoxon", groupby="louvain", use_raw=False)
overrepresentation_new = pt_enricher.hypergeometric(adata, targets=targets)

pt_enricher.score(adata, targets=targets)
enrichment_new = pt_enricher.gsea(adata, targets=targets)

## Comparison

In [4]:
# Check that the overrepresentations are the same
assert overrepresentation_old["B cells"].equals(overrepresentation_new["B cells"])

In [5]:
# Check that the enrichments are the same
all(k in enrichment_new and enrichment_old[k].equals(enrichment_new[k]) for k in enrichment_old) and len(
    enrichment_old
) == len(enrichment_new)

True